In [2]:
import numpy as np
#imports
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

## Detecção de Objetos 

Em visáo computacional, existem três tipos de aplicações principais: classificação, detecção e segmentação. Em classificação, dada uma imagem, a tarefa principal é reconhecer o objeto que está na imagem, como por exemplo, um gato. Por outro lado, caso o objetivo seja não apenas reconhecer o objeto como também localizá-lo na imagem, teremos um problema de detecção. Problemas de detecção, em geral, ao localizar uma imagem, incluem um retângulo (i.e. bounding box) que determina tudo que o algoritmo entende na imagem que represente este objeto. 

Ao adicionar as bounding boxes, percebe-se que fatalmente algumas porções do retângulo não representarão o objeto, a menos que o objeto também seja quadrado, Dessa forma, existe uma terceira aplicação, denominada segmentação, em que apenas os pixels que representam o objeto são selecionados. 

![alt text](../imgs/object_detection.jpeg "Title")


## Haar Cascade Classifier

Em visão computacional clássica, existe um classificador que utiliza algoritmo de aprendizagem de máquina (Adaboost) para análisar features da imagem.

### E o que são features?

Uma imagem é representada por uma matrix, sendo cada pixel um elemento desta matriz. A combinação de pixels semelhantes pode representar uma determinada característica de um objeto, denominada feature. Ao utilizar o Haar Cascade, o algoritmo percorre a matriz aplicando operações matemáticas (e.g. convolução) com a finalidade de identificar essas características e identificar as features. Ao realizar essas operações, o algoritmo identifica formas das features (i.e. linear, bordas horizontais, diagonais e verticais). 

![alt text](../imgs/lena.gif "Title")

Para reconhecimento facial, por exemplo, essas features podem ser olhos (formados por um conjunto de features horizontais), nariz (combinação de bordas verticais e diagonais) e boca (features horizontais).

![alt text](../imgs/features.png "Title")

Para identificar as features de forma que obtenha resultados satisfatórios e de forma rápida, o Haar possui diferentes estágios, sendo cada um deles uma parte da imagem com features (janela). Para cada janela, o classificador Adaboost é utilizado e, caso features correspondentes ao objeto sejam encontradas, a janela é utilizada, caso contrário, é rejeitada.

![alt text](../imgs/cascade.png "Title")




## Imagens e OpenCV

Como dito anteriormente, cada imagem é representada por uma matriz. Essa matriz poderá ser 2D caso a imagem seja binária, ou 3D caso a imagem seja em escala de cinza ou RGB. Para ambos os casos, as duas primeiras dimensões representam respectivamente linha e coluna da imagem e, para matrizes 3D, a terceira dimensão representará o canal de cor. Para imagens RGB, a terceira dimensão representará cada canal de cor (R = red, G = green e B = blue).   

In [ ]:
### Leitura de imagem em raw
img_raw = cv2.imread('../imgs/test/ceci.jpg')
print(type(img_raw)) 
print(img_raw.shape)
plt.imshow(img_raw)

In [ ]:
### Leitura de imagem RGB 
img = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
print(type(img_raw))
print(img_raw.shape)
plt.imshow(img)

In [ ]:
# Leitura de imagem em escaladas de Cinza
img = cv2.cvtColor(img_raw, cv2.COLOR_BGR2GRAY)
print(type(img_raw))
print(img_raw.shape)
plt.imshow(img, cmap='gray')

In [ ]:
### Exibindo imagens nos canais R, G e B
img_raw = cv2.imread('../imgs/test/ceci.jpg')
#Passo 1 - Crie uma cópia da imagem para cada canal
r = img_raw.copy()
b = img_raw.copy()
g = img_raw.copy()

#Passo 2 - Para cada canal, zere na matriz os canais que não serão utilizados
# para R, zere os canais 0 e 1.
r[:, :, 0] = 0
r[:, :, 1] = 0
# para G, zere canais 0 e 2 
g[:, :, 0] = 0
g[:, :, 2] = 0
# para B, zere os canais 1 e 2
b[:, :, 1] = 0
b[:, :, 2] = 0

# RGB - Blue
plt.imshow(b)
plt.show()
# RGB - Green
plt.imshow(g)
plt.show()
# RGB - Red
plt.imshow(r)
plt.show()

## Aplicando o Haar Cascade

Para aplicar o Haar, é necessário primeiramente obter um arquivo xml correspondente ao modelo do classificador obtido com o treinamento. O treino é a etapa em que o algoritmo irá analisar as imagens e compreender o que representa as features dos objetos. Essas imagens são dividas no mínimo em duas classes: i. imagens positivas (i.e. que representam o objeto a ser classificado) ii. imagens negativas (i.e. que representam qualquer outro objeto que não seja o objeto a ser classificado).

Existem duas formas de utilizar o classificador: utilizando um modelo pré-treinado ou treinando o próprio modelo. Para o modelo pré-treinado, o arquivo XML já estará gerado e a poderemos classificar novas imagens, enquanto para treinar o próprio modelo, o arquivo XML será gerado após o treinamento. 

O OpenCV disponibiliza modelos pré-treinados que podem ser utilizados para determinados objetos, entretanto, caso o objeto seja muito específico, um novo modelo deverá ser treinado. Para isso, uma base de dados de treino deverá ser gerada com os exemplos positivos e negativos balanceados. Para gerar a base, será necessário obter: i) as imagens, ii) a classe correspondente a imagem (e.g. 0 para rosto e 1 para não rosto), e iii) a localização do objeto, representada por quatro parâmetros (x,y,width, height). 

## Utilizando modelo pré-treinado do OpenCV

In [ ]:
### Passo 1 - Leitura do XML
face_cascade = cv2.CascadeClassifier('../data/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('../data/haarcascade_eye.xml')
### Passo 2 - Leitura da imagem
img = cv2.imread('../imgs/test/robot.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
### Passo 3 - Criar um objeto com o modelo cascade para a imagem. Isso é feito por meio do método
# detectMultiScale e possui três parâmetros: a imagem, o scaleFactor e o num_neighboors.
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
### Passo 4 - Identificar regiões de interesse (ROI) na imagem
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()